In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = ''

In [ ]:
df = pd.read_csv(base_path + 'featured_train_2.csv')

In [ ]:
df.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,matchType2,cat_numGroups
0,0,0,0.0,0,0,0,60,0,0.00,0,0.0000,0.00,0,244.80,1,0.4443,4,0
1,0,0,91.5,0,0,0,57,0,0.00,0,0.0045,11.04,0,1434.00,5,0.6400,4,0
2,1,0,68.0,0,0,0,47,0,0.00,0,0.0000,0.00,0,161.80,2,0.7754,2,1
3,0,0,32.9,0,0,0,75,0,0.00,0,0.0000,0.00,0,202.80,3,0.1667,4,0
4,0,0,100.0,0,0,0,45,1,58.53,0,0.0000,0.00,0,49.75,2,0.1875,1,2


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

feature = df.drop('winPlacePerc', axis=1)
target = df.winPlacePerc

# model = sm.OLS(target, feature).fit()
# print(1 / (1 - moㄹdel.rsquared))

# print(variance_inflation_factor(vif_df.values, 0))


pd.DataFrame({
    "VIF Factor": [variance_inflation_factor(df.values, idx) for idx in range(df.shape[1])],
    "features": df.columns,
})

,VIF Factor,features
0,1.492748,assists
1,3.329949,boosts
2,9.554946,damageDealt
3,3.577398,DBNOs
4,2.121225,headshotKills
5,1.881524,heals
6,13.887545,killPlace
7,9.664421,kills
8,2.074219,longestKill
9,1.312290,revives


데이터를 이용해 새로운 컬럼을 만드는 방법.

보통 이런경우 메치아이디로 그룹바이를 사용해 예측함

In [ ]:
# GBR_LightGBM_test 데이터를 합쳐서 분석함
# lightgbm-feature-engineering
df['headshotrate'] = df['kills']/df['headshotKills']
df['headshotKills_over_kills'] = df['headshotKills'] / df['kills']
df['killsPerWalkDistance'] = df['kills'] / df['walkDistance']
df['killStreakrate'] = df['killStreaks']/df['kills']


NameError: ignored

In [ ]:
# PUBG Finish Placement Prediction
# 이방법은 성능이 좋지 못함
df['all_distance']=df['rideDistance']+df['walkDistance']+df['swimDistance']
#Delete the old feature of distance 
df.drop(columns=['walkDistance','swimDistance','rideDistance'],axis=1,inplace=True)

df['medicine']=df['boosts']+df['heals']

df.drop(columns=['boosts','heals'],axis=1,inplace=True)

df.drop(columns=['Id','groupId','matchId','rankPoints','matchType'],inplace=True)

# 아웃라이어 제거
df.drop(df[df['medicine'] > 40].index, inplace=True)
df.drop(df[df['kills'] > 20].index, inplace=True)
df.drop(df[df['longestKill'] > 600].index, inplace=True)
df.drop(df[df['killStreaks']>12].index,inplace=True)
df.drop(df[df['weaponsAcquired'] >= 80].index, inplace=True)

In [ ]:
# PUBG Data Description A to Z + FE with Python 5. Feature Engineering부터 보면됨



기존컬럼만 사용하는 경우

In [ ]:
df[df.kills >= 8]['winPlacePerc'].mode()

0    1.0
dtype: float64

In [ ]:
df['kills'].quantile(0.99)

- 평균적으로 0.92킬을 합니다.

- 99퍼센트의 사람들이 7킬 이하를 합니다.

- 가장 많이 기록된 킬은 72킬입니다.

In [ ]:
# kills
# 8이상의 값이 전체 데이터의 1%이고
# 8이상 값의 타갯값(winPlacePerc)의 최빈값이 1이기 때문에 8이상의 값은 8로 취급힌디. 
# df.loc[df.kills >= 8,'kills'] = 8

In [ ]:
df.kills.value_counts()

In [ ]:
# weaponsAcquired을 킬과 연결시켜 전처리하는 코드는 없었음
# (하지만 거리와 연결시켜 전처히 하는 경우는 있음 ->gbr-lightgbm-test.ipynb)
df['distance_over_weapons'] = df['totalDistance'] / df['weaponsAcquired']
# 그 외의 언급은 거의 없음 -> 컬럼의 값을 그대로 사용함
# 굳이 전처리를 할 필요 없을 것 같으나 한다면 아래 방법 사용
df.drop(df[df['weaponsAcquired'] >= 80].index, inplace=True)

In [ ]:
# killstreaks
# pubg-prediction-beginner-s-practice-korean.ipynb 참조
# 킬한 사람 중에서는 1킬이 제일 많음 
# 연속킬중에서는 더블킬이 제일 많고 연속 3킬이 조금 존재하고 연속 4킬이 미세하게 존제함
# 연속 4킬 이상을 모두 4로 취급함 
df.loc[df.kills >= 4,'killstreaks'] = 4

# killstreaks에 대한 전처리과정은 없고 killstreaks 비율의 새로운 컬럼을 만들어서 머신러닝을 돌림
killStreak['killStreakrate'] = killStreak['killStreaks']/killStreak['kills']

In [ ]:
# damage Dealt는 전처리과정이 떠로 없고 
# '데이터를 이용해 새로운 컬럼을 만드는 방법.'에도
# 사용되지 않는다. 때문에 damage Dealt는 전처리를 하지 않는다. 


## 최종 결론
kills : 8이상을 8로 취합 : df.loc[df.kills >= 8,'kills'] = 8<br>
killstreaks : 연속 4킬 이상을 모두 4로 취급함 - df.loc[df.kills >= 4,'killstreaks'] = 4<br>
weaponsAcquired : 전처리 없음<br>
damage Dealt : 전처리 없음<br>